In [16]:
import requests
import json
from newsapi import NewsApiClient

In [17]:
# getting our API keys without putting them on GitHub for you dirty theives to steal
with open('keys.json', 'r') as f:
    keys_dict = json.load(f)
newsapi_key = keys_dict["newsapi"]
summerizebot_key = keys_dict["summerizebot"]
summerize_request = "https://www.summarizebot.com/api/extract?apiKey=" + summerizebot_key + "&url={}"

In [67]:
# input: article search query as a string
# does: aggregates articles based on query, then stores those articles as JSON files
def getArticles( query ):
    # get the articles with News API
    client = NewsApiClient(api_key=newsapi_key)
    articles = client.get_everything(q=query,
                                      from_param='2019-09-20',
                                      to='2019-10-15',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
    '''
    for article in articles["articles"]:
        print( article )
    '''
    
    #  load our list of previously downloaded files
    with open('articles/files.txt', 'r') as f:
        downloaded_files = f.readlines()
        downloaded_files = [file[0:len(file) - 1] for file in downloaded_files]
    
    # download any new ones
    
    for article in articles["articles"]:
        if article["url"] not in downloaded_files:
            # always mark that we've processed the file after to prevent re-requests
            with open('articles/files.txt', 'a') as f:
                f.write("" + article["url"] + "\n")
            
            print("making request")
            articleData = requests.get(summerize_request.format(article["url"]))
            
            # make sure the file is valid
            if '/' not in articleData.json()['article title']:
                filename = 'articles/' + articleData.json()['article title'] + '.json'
                print( articleData.json() )
                with open(filename, 'w') as fp:
                    json.dump(articleData.json(), fp)

In [68]:
getArticles('beijing')